# 🎓 Введение

Привет, участники соревнования! 👋

Этот ноутбук — ваш проводник в мир анализа временных рядов и построения моделей машинного обучения. Он написан специально для тех, кто только начинает знакомство с Kaggle и участвует в соревнованиях впервые.

Здесь мы вместе:
- 📈 научимся извлекать признаки из временных рядов,
- 🧠 построим базовую модель машинного обучения (LightGBM),
- 🧪 проведём корректную валидацию,
- 📤 подготовим прогноз и отправим его в виде submission-файла.

> 💡 Несмотря на то, что текущее решение не побеждает бенчмарки, **совсем небольшие улучшения** (например, другие признаки, параметры модели или архитектура) уже позволят вам занять **топовые места** в рейтинге!

Цель этого ноутбука — **показать вам базовый, но правильный путь**:
- никаких "магических" трюков,
- только понятные и воспроизводимые шаги.

Удачи в соревновании! 🚀  
**Вы уже на правильном пути.**


In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os

# 1. Загрузка
# work_dir = "../kaggle_data_ts/"
work_dir = "../input/kazakhstan-ai-respa-take-home/"
train = pd.read_csv(os.path.join(work_dir, "train.csv"))
train['submitted_date'] = pd.to_datetime(train['submitted_date'])
train.head()

,category,submitted_date,num_papers
0,hep-ph - High Energy Physics - Phenomenology,2000-01-01,5
1,hep-ph - High Energy Physics - Phenomenology,2000-01-02,6
2,hep-ph - High Energy Physics - Phenomenology,2000-01-03,14
3,hep-ph - High Energy Physics - Phenomenology,2000-01-04,10
4,hep-ph - High Energy Physics - Phenomenology,2000-01-05,16


## 📂 Работа с одной категорией

Для начала сосредоточимся на одной подкатегории. Мы извлечём её временной ряд — упорядочим по дате, сделаем дату индексом и уберём лишние колонки. Это поможет нам удобно считать rolling-статистики и готовить признаки.


In [2]:
def get_category_data(df: pd.DataFrame, category: str):
    category_data = df[df['category'] == category].copy()
    category_data = category_data.sort_values('submitted_date')
    category_data = category_data.set_index('submitted_date')
    return category_data.drop('category', axis=1)

category_data = get_category_data(train, 'hep-ph - High Energy Physics - Phenomenology')
category_data

,num_papers
submitted_date,
2000-01-01,5
2000-01-02,6
2000-01-03,14
2000-01-04,10
2000-01-05,16
...,...
2025-02-05,32
2025-02-06,29
2025-02-07,25


## 🔮 Расширение данных на будущее

Чтобы наша модель могла делать прогнозы, мы расширим временной ряд на одну неделю вперёд.  

Мы добавим 7 будущих дат без значений — они нужны, чтобы модель могла "заглянуть в будущее" и сгенерировать признаки для предсказания тестовых данных.

Для простоты мы будем прогнозировать только одну следующую неделю - а дальше мы будем просто повторять наши прогнозы

In [3]:
def extend_dataset(category_data, last_train_date, future_weeks_num: int = 0):
    if future_weeks_num > 0:
        future_dates = pd.date_range(
                start=last_train_date + pd.Timedelta(days=1),
                periods=future_weeks_num * 7,
                freq='D'
            )
            
        future = pd.DataFrame(index= future_dates, data={'num_papers': None})
        return pd.concat((category_data, future))

last_train_date=train.submitted_date.max()
category_data = extend_dataset(category_data, last_train_date=last_train_date, future_weeks_num=1)

## 📊 Расчёт rolling-признаков (скользящих аггрегаций)

Теперь посчитаем базовые признаки по скользящему окну (rolling window):

- `rolling_sum_during_week` — сколько статей было опубликовано за последние 7 дней.
- `rolling_max_during_week` — максимум статей этой категории в день за последние 7 дней.
- `rolling_min_during_month` и `rolling_max_during_month` — минимум и максимум за последние 28 дней (месяц).

Эти признаки помогут модели понимать недавнюю активность в категории.


In [4]:
def get_rolling_features(cat_data: pd.DataFrame):
    daily_features = pd.DataFrame(index=cat_data.index)
    # Weekly rolling features
    weekly_roll = cat_data['num_papers'].rolling('7D', min_periods=1)
    daily_features['rolling_sum_during_week'] = weekly_roll.sum()
    daily_features['rolling_max_during_week'] = weekly_roll.max()
    daily_features['rolling_min_during_week'] = weekly_roll.min()
    daily_features['rolling_mean_during_week'] = weekly_roll.mean()
    daily_features['rolling_std_during_week'] = weekly_roll.std()

    # Monthly rolling features
    month_roll = cat_data['num_papers'].rolling('28D', min_periods=1)
    daily_features['rolling_min_during_month'] = month_roll.min()
    daily_features['rolling_max_during_month'] = month_roll.max()
    daily_features['rolling_mean_during_month'] = month_roll.mean()
    daily_features['rolling_std_during_month'] = month_roll.std()


    return daily_features

rolling_features = get_rolling_features(cat_data=category_data)
rolling_features

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month
2000-01-01,5.0,5.0,5.0,5.000000,NaN,5.0,5.0,5.000000,NaN
2000-01-02,11.0,6.0,5.0,5.500000,0.707107,5.0,6.0,5.500000,0.707107
2000-01-03,25.0,14.0,5.0,8.333333,4.932883,5.0,14.0,8.333333,4.932883
2000-01-04,35.0,14.0,5.0,8.750000,4.112988,5.0,14.0,8.750000,4.112988
2000-01-05,51.0,16.0,5.0,10.200000,4.816638,5.0,16.0,10.200000,4.816638
...,...,...,...,...,...,...,...,...,...
2025-02-12,65.0,29.0,5.0,16.250000,12.526638,5.0,32.0,20.360000,8.966976
2025-02-13,36.0,25.0,5.0,12.000000,11.269428,5.0,32.0,20.208333,9.127022
2025-02-14,11.0,6.0,5.0,5.500000,0.707107,5.0,32.0,19.826087,9.133615
2025-02-15,6.0,6.0,6.0,6.000000,NaN,5.0,32.0,20.409091,8.899754


In [5]:
def add_interaction_features(features: pd.DataFrame):
    # Interaction between rolling mean and standard deviation
    features['interaction_mean_std_week'] = features['rolling_mean_during_week'] * features['rolling_std_during_week']
    features['interaction_sum_max_week'] = features['rolling_sum_during_week'] * features['rolling_max_during_week']
    features['interaction_min_max_week'] = features['rolling_max_during_week'] - features['rolling_min_during_week']
  
    return features


## ⏪ Добавление лагов (значений сдвига)

Теперь добавим **лаг-признаки** — значения, которые были ровно неделю назад:

- Например, `rolling_sum_during_week_last_week` — это сумма за неделю, но из **предыдущей** недели.
- Такие признаки важны, чтобы модель могла "вспоминать", что происходило ранее, и делать выводы на основе динамики.

Мы используем `.shift(freq='7D')`, чтобы сместить значения на 7 дней назад.

In [6]:
def add_lag_features(features: pd.DataFrame):
    new_features = features.copy()
    for col in features.columns:
        # Add lag features for multiple weeks
        for lag in [1, 2, 4, 8, 14]:  # You can adjust these values as needed
            new_features[f'{col}_lag_{lag}_week'] = features[col].shift(freq=pd.Timedelta(days=7*lag))
    return new_features.dropna()

lag_features = add_lag_features(rolling_features)
lag_features

,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month,rolling_sum_during_week_lag_1_week,...,rolling_mean_during_month_lag_1_week,rolling_mean_during_month_lag_2_week,rolling_mean_during_month_lag_4_week,rolling_mean_during_month_lag_8_week,rolling_mean_during_month_lag_14_week,rolling_std_during_month_lag_1_week,rolling_std_during_month_lag_2_week,rolling_std_during_month_lag_4_week,rolling_std_during_month_lag_8_week,rolling_std_during_month_lag_14_week
2000-04-09,101.0,22.0,5.0,14.428571,6.900656,0.0,27.0,13.607143,7.410400,102.0,...,13.250000,12.285714,12.357143,14.535714,5.500000,7.520958,7.080789,6.940030,7.965119,0.707107
2000-04-10,100.0,22.0,5.0,14.285714,6.872998,0.0,27.0,13.607143,7.410400,104.0,...,13.214286,12.357143,12.035714,14.464286,8.333333,7.504849,7.087884,6.579892,7.866865,4.932883
2000-04-11,90.0,22.0,5.0,12.857143,5.984106,0.0,27.0,13.428571,7.385815,108.0,...,13.535714,12.392857,12.035714,14.357143,8.750000,7.685880,7.114613,6.579892,7.808895,4.112988
2000-04-12,93.0,22.0,5.0,13.285714,5.936168,0.0,27.0,13.250000,7.306136,106.0,...,13.428571,12.357143,12.321429,14.000000,10.200000,7.700065,7.108755,6.705739,7.722022,4.816638
2000-04-13,87.0,18.0,5.0,12.428571,4.790864,0.0,27.0,13.035714,7.125760,108.0,...,13.428571,12.785714,12.464286,14.000000,10.333333,7.700065,7.197516,6.871746,7.722022,4.320494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-10,120.0,32.0,5.0,20.000000,11.661904,5.0,32.0,20.740741,8.733633,133.0,...,19.892857,19.000000,22.178571,25.071429,22.500000,7.814229,8.036952,13.952442,11.237927,12.106013
2025-02-11,97.0,32.0,5.0,19.400000,12.934450,5.0,32.0,20.500000,8.814760,134.0,...,20.035714,19.000000,21.821429,25.464286,22.392857,7.833840,8.036952,13.684415,11.461634,12.059541
2025-02-12,65.0,29.0,5.0,16.250000,12.526638,5.0,32.0,20.360000,8.966976,137.0,...,20.357143,19.607143,20.535714,26.464286,22.142857,8.138679,8.130142,11.477779,13.150080,11.881247
2025-02-13,36.0,25.0,5.0,12.000000,11.269428,5.0,32.0,20.208333,9.127022,138.0,...,20.714286,20.035714,19.821429,27.178571,21.928571,8.294832,8.248376,10.548139,13.548414,11.806867


## 📆 Преобразование в недельный уровень

Поскольку нам нужно делать прогноз **по неделям**, агрегируем ежедневные признаки в недельные.

- Каждому дню мы сопоставим конец недели (воскресенье).
- Затем для каждой недели возьмём **последнее доступное значение** признаков.
- Это даст нам ровно одну строку признаков на каждую неделю.

Теперь мы готовы строить таргет и обучать модель.


In [7]:
def build_weekly_features(features):
    daily_features = features.reset_index(names="day")
    daily_features['week'] = daily_features['day'] + pd.to_timedelta(6 - daily_features['day'].dt.weekday, unit='D')
    weekly_features = daily_features.groupby('week').last().reset_index()
    return weekly_features.drop('day', axis=1)

weekly_features = build_weekly_features(lag_features)
weekly_features

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month,...,rolling_mean_during_month_lag_1_week,rolling_mean_during_month_lag_2_week,rolling_mean_during_month_lag_4_week,rolling_mean_during_month_lag_8_week,rolling_mean_during_month_lag_14_week,rolling_std_during_month_lag_1_week,rolling_std_during_month_lag_2_week,rolling_std_during_month_lag_4_week,rolling_std_during_month_lag_8_week,rolling_std_during_month_lag_14_week
0,2000-04-09,101.0,22.0,5.0,14.428571,6.900656,0.0,27.0,13.607143,7.410400,...,13.250000,12.285714,12.357143,14.535714,5.500000,7.520958,7.080789,6.940030,7.965119,0.707107
1,2000-04-16,83.0,16.0,6.0,11.857143,3.670993,1.0,27.0,13.321429,6.324869,...,13.607143,13.250000,12.214286,14.285714,8.666667,7.410400,7.520958,7.325031,7.615078,4.716991
2,2000-04-23,73.0,21.0,0.0,10.428571,7.590721,0.0,27.0,12.821429,6.700213,...,13.321429,13.607143,12.285714,13.464286,11.437500,6.324869,7.410400,7.080789,7.805421,7.004463
3,2000-04-30,93.0,26.0,3.0,13.285714,8.635475,0.0,26.0,12.500000,6.730252,...,12.821429,13.321429,13.250000,12.535714,12.391304,6.700213,6.324869,7.520958,7.396106,7.626146
4,2000-05-07,66.0,15.0,0.0,9.428571,5.740416,0.0,26.0,11.250000,6.472878,...,12.500000,12.821429,13.607143,12.357143,13.642857,6.730252,6.700213,7.410400,6.940030,7.414771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2025-01-19,145.0,29.0,7.0,20.714286,8.596788,4.0,47.0,19.321429,9.963093,...,22.785714,25.535714,27.785714,23.392857,21.178571,14.397420,15.014763,13.639152,11.798742,10.029784
1294,2025-01-26,157.0,31.0,12.0,22.428571,7.091242,4.0,47.0,19.642857,9.460304,...,19.321429,22.785714,26.892857,23.142857,21.392857,9.963093,14.397420,13.706823,10.609968,10.485755
1295,2025-02-02,139.0,29.0,7.0,19.857143,9.441550,7.0,31.0,20.142857,8.026411,...,19.642857,19.321429,25.535714,23.928571,20.785714,9.460304,9.963093,15.014763,10.659472,11.159972
1296,2025-02-09,143.0,32.0,5.0,20.428571,10.706029,5.0,32.0,20.857143,8.592479,...,20.142857,19.642857,22.785714,25.142857,22.321429,8.026411,9.460304,14.397420,11.348729,12.012505


## 🎯 Построение целевой переменной (target)

Теперь создадим **таргет** — количество публикаций в каждой неделе, сдвинутое на `week_horizon`.

- Если `week_horizon=1`, мы будем предсказывать число публикаций **на следующей неделе**.
- Мы агрегируем данные по неделям (`resample('W')`) и сдвигаем их вверх (`shift(-1)`), чтобы каждая неделя "смотрела в будущее".

Это наша целевая переменная для обучения модели.


In [8]:
def build_targets(category_data, week_horizon: int):
    targets = category_data.resample('W').sum().shift(-week_horizon).num_papers.rename('target')
    targets.index.name = 'week'
    return targets

targets = build_targets(category_data=category_data, week_horizon=1)
targets

week
2000-01-02      67
2000-01-09     105
2000-01-16     102
2000-01-23     108
2000-01-30     101
              ... 
2025-01-19     157
2025-01-26     139
2025-02-02     143
2025-02-09       0
2025-02-16    None
Freq: W-SUN, Name: target, Length: 1312, dtype: object

# Итак по одной категории у нас уже есть готовый датасет для тренировки!

In [9]:
current_data = weekly_features.merge(targets, on='week')
current_data

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month,...,rolling_mean_during_month_lag_2_week,rolling_mean_during_month_lag_4_week,rolling_mean_during_month_lag_8_week,rolling_mean_during_month_lag_14_week,rolling_std_during_month_lag_1_week,rolling_std_during_month_lag_2_week,rolling_std_during_month_lag_4_week,rolling_std_during_month_lag_8_week,rolling_std_during_month_lag_14_week,target
0,2000-04-09,101.0,22.0,5.0,14.428571,6.900656,0.0,27.0,13.607143,7.410400,...,12.285714,12.357143,14.535714,5.500000,7.520958,7.080789,6.940030,7.965119,0.707107,83
1,2000-04-16,83.0,16.0,6.0,11.857143,3.670993,1.0,27.0,13.321429,6.324869,...,13.250000,12.214286,14.285714,8.666667,7.410400,7.520958,7.325031,7.615078,4.716991,73
2,2000-04-23,73.0,21.0,0.0,10.428571,7.590721,0.0,27.0,12.821429,6.700213,...,13.607143,12.285714,13.464286,11.437500,6.324869,7.410400,7.080789,7.805421,7.004463,93
3,2000-04-30,93.0,26.0,3.0,13.285714,8.635475,0.0,26.0,12.500000,6.730252,...,13.321429,13.250000,12.535714,12.391304,6.700213,6.324869,7.520958,7.396106,7.626146,66
4,2000-05-07,66.0,15.0,0.0,9.428571,5.740416,0.0,26.0,11.250000,6.472878,...,12.821429,13.607143,12.357143,13.642857,6.730252,6.700213,7.410400,6.940030,7.414771,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2025-01-19,145.0,29.0,7.0,20.714286,8.596788,4.0,47.0,19.321429,9.963093,...,25.535714,27.785714,23.392857,21.178571,14.397420,15.014763,13.639152,11.798742,10.029784,157
1294,2025-01-26,157.0,31.0,12.0,22.428571,7.091242,4.0,47.0,19.642857,9.460304,...,22.785714,26.892857,23.142857,21.392857,9.963093,14.397420,13.706823,10.609968,10.485755,139
1295,2025-02-02,139.0,29.0,7.0,19.857143,9.441550,7.0,31.0,20.142857,8.026411,...,19.321429,25.535714,23.928571,20.785714,9.460304,9.963093,15.014763,10.659472,11.159972,143
1296,2025-02-09,143.0,32.0,5.0,20.428571,10.706029,5.0,32.0,20.857143,8.592479,...,19.642857,22.785714,25.142857,22.321429,8.026411,9.460304,14.397420,11.348729,12.012505,0


## 🏗️ Сбор финального датасета для обучения

Теперь объединим всё вместе для всех категорий:

1. Для каждой категории:
   - Извлекаем временной ряд.
   - Добавляем будущую неделю для генерации признаков.
   - Считаем rolling-признаки.
   - Добавляем лаги (предыдущие недели).
   - Агрегируем всё на недельном уровне.
   - Создаём `target` — количество публикаций на следующей неделе.
2. Объединяем данные всех категорий в один финальный `dataset`.
3. Преобразуем колонку `category` в категориальный тип (для LightGBM).

Теперь у нас есть полноценный обучающий набор, готовый для машинного обучения! 💪


In [10]:
from tqdm.auto import tqdm

last_train_date=train.submitted_date.max()
progress_bar = tqdm(train.category.unique())

dataset = []
for category in progress_bar:
    category_data = get_category_data(train, category)
    extended_category_data = extend_dataset(category_data, last_train_date=last_train_date, future_weeks_num=1)
    rolling_features = get_rolling_features(cat_data=extended_category_data)
    rolling_features = add_interaction_features(rolling_features)
    lag_features = add_lag_features(rolling_features)
    weekly_features = build_weekly_features(lag_features)
    targets = build_targets(category_data=category_data, week_horizon=1)
    data = weekly_features.merge(targets, on='week')
    data['category'] = category
    dataset.append(data)

dataset = pd.concat(dataset)
dataset['category'] = dataset['category'].astype('category')
dataset

  0%|          | 0/140 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month,...,interaction_sum_max_week_lag_4_week,interaction_sum_max_week_lag_8_week,interaction_sum_max_week_lag_14_week,interaction_min_max_week_lag_1_week,interaction_min_max_week_lag_2_week,interaction_min_max_week_lag_4_week,interaction_min_max_week_lag_8_week,interaction_min_max_week_lag_14_week,target,category
0,2000-04-09,101.0,22.0,5.0,14.428571,6.900656,0.0,27.0,13.607143,7.410400,...,2002.0,2304.0,66.0,26.0,16.0,20.0,23.0,1.0,83.0,hep-ph - High Energy Physics - Phenomenology
1,2000-04-16,83.0,16.0,6.0,11.857143,3.670993,1.0,27.0,13.321429,6.324869,...,2002.0,2280.0,1072.0,17.0,26.0,22.0,21.0,14.0,73.0,hep-ph - High Energy Physics - Phenomenology
2,2000-04-23,73.0,21.0,0.0,10.428571,7.590721,0.0,27.0,12.821429,6.700213,...,1479.0,1785.0,2520.0,10.0,17.0,16.0,19.0,21.0,93.0,hep-ph - High Energy Physics - Phenomenology
3,2000-04-30,93.0,26.0,3.0,13.285714,8.635475,0.0,26.0,12.500000,6.730252,...,2754.0,1350.0,2652.0,21.0,10.0,26.0,16.0,24.0,66.0,hep-ph - High Energy Physics - Phenomenology
4,2000-05-07,66.0,15.0,0.0,9.428571,5.740416,0.0,26.0,11.250000,6.472878,...,2222.0,2002.0,2592.0,23.0,21.0,17.0,20.0,16.0,94.0,hep-ph - High Energy Physics - Phenomenology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,2025-01-12,62.0,11.0,7.0,8.857143,1.573592,5.0,24.0,10.285714,4.814001,...,1824.0,1326.0,3752.0,6.0,10.0,12.0,12.0,22.0,65.0,q-bio
1293,2025-01-19,65.0,15.0,4.0,9.285714,4.309458,4.0,15.0,9.107143,3.059161,...,2352.0,1960.0,1615.0,4.0,6.0,19.0,14.0,13.0,79.0,q-bio
1294,2025-01-26,79.0,15.0,8.0,11.285714,2.288689,4.0,15.0,9.500000,2.899553,...,1020.0,1628.0,3080.0,11.0,4.0,10.0,19.0,21.0,84.0,q-bio
1295,2025-02-02,84.0,20.0,3.0,12.000000,5.597619,3.0,20.0,10.357143,3.822102,...,660.0,1656.0,2472.0,7.0,11.0,6.0,12.0,16.0,90.0,q-bio


## 🧪 Разделение на обучающую, валидационную и тестовую выборки

Перед обучением модели нужно правильно разделить данные:

- **train_dataset** — всё до последних 4 недель (обучение).
- **valid_dataset** — последние 4 недели перед последней известной датой (валидация).
- **test_dataset** — данные без таргета (будущее, на которое мы будем делать предсказание).

Это разделение имитирует реальную ситуацию: мы учимся на прошлом, проверяем на последнем известном фрагменте, и делаем прогноз на будущее.


In [11]:
labeled_data = dataset[dataset.target.notnull()].reset_index(drop=True).dropna()

n_valid_weeks = 4
valid_start_date = last_train_date - pd.Timedelta(days=7 * n_valid_weeks)

print(f"Предпоследняя неделя: {valid_start_date}")
valid_dataset = labeled_data[labeled_data.week >= valid_start_date]
train_dataset = labeled_data[labeled_data.week < valid_start_date]
test_dataset = dataset[dataset.target.isnull()].reset_index(drop=True)

print(train_dataset.shape, valid_dataset.shape, test_dataset.shape)

Предпоследняя неделя: 2025-01-12 00:00:00
(173524, 75) (560, 75) (140, 75)


## 📦 Подготовка данных для LightGBM

Теперь подготовим данные в нужном формате для обучения модели:

- `train_set` и `valid_set` создаются в формате `lightgbm.Dataset` — это специальный формат для ускорения обучения.
- `test_set` остаётся обычным `DataFrame`, потому что мы просто хотим получить предсказания.

Колонку `target` мы используем только как целевую переменную (`label`).


In [12]:
import lightgbm

train_set = lightgbm.Dataset(train_dataset.drop(['week', 'target'], axis=1), label=train_dataset['target'])
valid_set = lightgbm.Dataset(valid_dataset.drop(['week', 'target'], axis=1), label=valid_dataset['target'])
test_set = test_dataset.drop(['week', 'target'], axis=1)

## 🌲 Обучение модели LightGBM с кастомной метрикой

Теперь мы обучим модель LightGBM для задачи регрессии:

- Используем метрику `Safe MAPE` — она помогает избежать слишком большого штрафа на малых значениях.
- Указываем параметры модели:
  - `objective: regression` — мы предсказываем количество статей (то есть числа).
  - `learning_rate: 0.05` — насколько быстро обучается модель.
  - `depth: 5` — максимальная глубина дерева.
  - `metric: None` — мы используем свою метрику, не встроенную.

Мы также включаем:
- **Раннюю остановку** (`early_stopping`), чтобы не переобучиться.
- **Логгинг** каждые 50 итераций, чтобы отслеживать процесс обучения.

Параметры тренировки были выбраны случайно, просто чтобы натренировать хоть что-то

In [13]:
def safe_mape_lgb(y_pred, dataset):
    y_true = dataset.get_label()
    denominator = pd.Series(y_true).abs().clip(lower=10.0)
    error = abs(y_pred - y_true) / denominator
    return 'safe_mape', error.mean(), False  # False = the lower the better

# 2. Параметры модели
params = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'depth': 5,
    'verbosity': -1,
    'metric': 'None'
}

# 3. Обучение с кастомной метрикой
model = lightgbm.train(
    params,
    train_set,
    num_boost_round=200,
    valid_sets=[valid_set],
    valid_names=['valid'],
    feval=safe_mape_lgb,
    callbacks=[
        lightgbm.early_stopping(stopping_rounds=50),
        lightgbm.log_evaluation(period=50)
    ]
)


Training until validation scores don't improve for 50 rounds
[50]	valid's safe_mape: 0.210672
[100]	valid's safe_mape: 0.191957
[150]	valid's safe_mape: 0.191889
Early stopping, best iteration is:
[113]	valid's safe_mape: 0.191743


# Используем нашу натренированную модель на тестовых данных

In [14]:
test_dataset['predicted'] = model.predict(test_set)
test_dataset[['category', 'predicted']]

,category,predicted
0,hep-ph - High Energy Physics - Phenomenology,146.699529
1,math.CO - Combinatorics,107.097208
2,cs.CG - Computational Geometry,10.062030
3,physics.gen-ph - General Physics,5.344321
4,math.CA - Classical Analysis and ODEs,30.419069
...,...,...
135,eess.IV - Image and Video Processing,89.486911
136,eess.SP - Signal Processing,109.498973
137,q-fin.MF - Mathematical Finance,5.635077
138,cond-mat,416.645681


# Делаем финальный сабмишен

проверяем что наш submission упорядочен также как sample_submission.csv

In [15]:

sample_submission = pd.read_csv(os.path.join(work_dir, "sample_submission.csv"))
sample_submission['category'] = sample_submission['id'].apply(lambda x: x.split('__')[0])
sample_submission = sample_submission.merge(test_dataset[['category', 'predicted']], on='category')
sample_submission[['id', 'predicted']].to_csv('submission.csv', index=False)